# Clustering Neighborhoods in Toronto
### Assignment Week 3 - Applied Data Science Capstone - Markus Kessler

## Web Scraping

In [58]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
# send requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
# read out table

data = soup.tbody.find_all('tr')
table = []

for item in data:
    table.append(item.text.split('\n'))

raw_df = pd.DataFrame(table)
raw_df

,0,1,2,3,4,5,6
0,,Postal code,,Borough,,Neighborhood,
1,,M1A,,Not assigned,,,
2,,M2A,,Not assigned,,,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
...,...,...,...,...,...,...,...
176,,M5Z,,Not assigned,,,
177,,M6Z,,Not assigned,,,
178,,M7Z,,Not assigned,,,
179,,M8Z,,Etobicoke,,Mimico NW / The Queensway West / South of Bloo...,


In [5]:
#  preprocess the data
cpc_df = raw_df.copy()
cpc_df.drop(columns=[0, 2, 4, 6], inplace=True)
new_header = [cpc_df.iloc[0]]
cpc_df = cpc_df[1:]
cpc_df.columns = ['Postal Code', 'Borough', 'Neighborhood']

filter = cpc_df[['Borough']] != 'Not assigned'
cpc_df = cpc_df[filter['Borough']]
cpc_df



,Postal Code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
161,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,Business reply mail Processing CentrE
170,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


## Geographical Data

In [6]:
'''import geocoder # import geocoder

lat_lng_coords = None
latitudes = []
longitudes = []

for pc on cpc_df['Postal codes']
    while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(pc))
    lat_lng_coords = g.latlng

    latitudes.append(lat_lng_coords[0])
    longitudes.append(lat_lng_coords[1])

cpc_df['Latitudes'] = latitudes
cpc_df['Longitudes'] = longitudes'''
# geocoder not working, keeps loading forever on the first coordinate

"import geocoder # import geocoder\n\nlat_lng_coords = None\nlatitudes = []\nlongitudes = []\n\nfor pc on cpc_df['Postal codes']\n    while(lat_lng_coords is None):\n    g = geocoder.google('{}, Toronto, Ontario'.format(pc))\n    lat_lng_coords = g.latlng\n\n    latitudes.append(lat_lng_coords[0])\n    longitudes.append(lat_lng_coords[1])\n\ncpc_df['Latitudes'] = latitudes\ncpc_df['Longitudes'] = longitudes"

In [7]:
# Downloaded the file, as geocoder didn't seem to work

coord_df = pd.read_csv(r'C:\Users\Markus\OneDrive\Arbeit\Data Science\Coursera Notebooks\Coursera_Capstone\Geospatial_coordinates.csv')
coord_df #zip_df = cpc_df.zip(coord_df)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [8]:
# merging the two dataframes together
final_df = pd.merge(cpc_df, coord_df, on='Postal Code', how='outer')
final_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509


## Clustering the Neighbourhoods

In [9]:
# Filtering all boroughs containing 'Toronto' in their name
toronto_df = final_df[final_df['Borough'].str.contains('Toronto')].copy()
toronto_df.drop(columns=['Borough'], inplace=True)
toronto_df



,Postal Code,Neighborhood,Latitude,Longitude
2,M5A,Regent Park / Harbourfront,43.654260,-79.360636
4,M7A,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,M5B,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,St. James Town,43.651494,-79.375418
19,M4E,The Beaches,43.676357,-79.293031
20,M5E,Berczy Park,43.644771,-79.373306
24,M5G,Central Bay Street,43.657952,-79.387383
25,M6G,Christie,43.669542,-79.422564
30,M5H,Richmond / Adelaide / King,43.650571,-79.384568
31,M6H,Dufferin / Dovercourt Village,43.669005,-79.442259


In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.65, -79.39], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [68]:
CLIENT_ID = '*******' # your Foursquare ID
CLIENT_SECRET = '*********' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *******
CLIENT_SECRET:*********


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
result = getNearbyVenues(toronto_df['Neighborhood'], toronto_df['Latitude'], toronto_df['Longitude'])
result

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Roseda

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
...,...,...,...,...,...,...,...
857,Business reply mail Processing CentrE,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
858,Business reply mail Processing CentrE,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
859,Business reply mail Processing CentrE,43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
860,Business reply mail Processing CentrE,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station


In [41]:
toronto_onehot = pd.get_dummies(result['Venue Category'],  prefix="", prefix_sep="")
toronto_onehot['Name'] = result['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Name').mean().reset_index()
toronto_grouped

,Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,CN Tower / King and Spadina / Railway Lands / ...,0.066667,0.066667,0.066667,0.133333,0.2,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333
5,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Commerce Court / Victoria Hotel,0.000000,0.000000,0.000000,0.000000,0.0,0.033333,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
num_top_venues = 10

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Name'] = toronto_grouped['Name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Café,Cocktail Bar,Farmers Market,Beer Bar,Bakery,Park,Liquor Store,Breakfast Spot
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Nightclub,Breakfast Spot,Climbing Gym,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Burrito Place
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport,Bar,Harbor / Marina,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Coffee Shop
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Spa,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Park,Modern European Restaurant


In [56]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Name'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Regent Park / Harbourfront,43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Dessert Shop,Chocolate Shop,Event Space,Pub,Spa
4,M7A,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Juice Bar,Italian Restaurant,Burger Joint,Burrito Place,Discount Store,Spa
9,M5B,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Coffee Shop,Clothing Store,College Rec Center,Hotel,Bakery,Sporting Goods Shop,Electronics Store,Shopping Mall,Mexican Restaurant
15,M5C,St. James Town,43.651494,-79.375418,4,Gastropub,Café,Coffee Shop,Hotel,Farmers Market,Gym,Japanese Restaurant,Italian Restaurant,Diner,Ice Cream Shop
19,M4E,The Beaches,43.676357,-79.293031,0,Neighborhood,Trail,Pub,Health Food Store,Yoga Studio,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner


## Visualizing the data

In [59]:
# create map
map_clusters = folium.Map(location=[43.65, -79.39], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the results

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]


,Postal Code,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,M6H,Dufferin / Dovercourt Village,43.669005,-79.442259,1,Pharmacy,Bakery,Music Venue,Brazilian Restaurant,Café,Recording Studio,Bar,Bank,Supermarket,Brewery
37,M6J,Little Portugal / Trinity,43.647927,-79.419750,1,Bar,Vietnamese Restaurant,Asian Restaurant,Men's Store,Yoga Studio,New American Restaurant,Bistro,Boutique,Brewery,Record Shop
41,M4K,The Danforth West / Riverdale,43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Cosmetics Shop,Pizza Place,Dessert Shop,Pub,Diner,Bakery,Café
43,M6K,Brockton / Parkdale Village / Exhibition Place,43.636847,-79.428191,1,Café,Coffee Shop,Nightclub,Breakfast Spot,Climbing Gym,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
54,M4M,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Brewery,Bar,Ice Cream Shop,Bookstore,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
68,M5P,Forest Hill North & West,43.696948,-79.411307,1,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Deli / Bodega,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
69,M6P,High Park / The Junction South,43.661608,-79.464763,1,Mexican Restaurant,Café,Thai Restaurant,Park,Speakeasy,Bookstore,Cajun / Creole Restaurant,Music Venue,Diner,Bakery
75,M6R,Parkdale / Roncesvalles,43.648960,-79.456325,1,Gift Shop,Bar,Coffee Shop,Movie Theater,Cuban Restaurant,Dessert Shop,Restaurant,Dog Run,Eastern European Restaurant,Breakfast Spot
80,M5S,University of Toronto / Harbord,43.662696,-79.400049,1,Café,Bookstore,Bakery,Restaurant,Japanese Restaurant,Italian Restaurant,Bar,Comfort Food Restaurant,College Arts Building,Nightclub
81,M6S,Runnymede / Swansea,43.651571,-79.484450,1,Café,Coffee Shop,Sushi Restaurant,Italian Restaurant,Indie Movie Theater,Juice Bar,Bar,Falafel Restaurant,Bookstore,Sandwich Place


In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Postal Code,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Regent Park / Harbourfront,43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Dessert Shop,Chocolate Shop,Event Space,Pub,Spa
4,M7A,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Juice Bar,Italian Restaurant,Burger Joint,Burrito Place,Discount Store,Spa
9,M5B,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Coffee Shop,Clothing Store,College Rec Center,Hotel,Bakery,Sporting Goods Shop,Electronics Store,Shopping Mall,Mexican Restaurant
15,M5C,St. James Town,43.651494,-79.375418,4,Gastropub,Café,Coffee Shop,Hotel,Farmers Market,Gym,Japanese Restaurant,Italian Restaurant,Diner,Ice Cream Shop
20,M5E,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Seafood Restaurant,Café,Cocktail Bar,Farmers Market,Beer Bar,Bakery,Park,Liquor Store,Breakfast Spot
24,M5G,Central Bay Street,43.657952,-79.387383,4,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Spa,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Park,Modern European Restaurant
25,M6G,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Athletics & Sports,Coffee Shop,Candy Store,Restaurant,Diner,Italian Restaurant,Baby Store
30,M5H,Richmond / Adelaide / King,43.650571,-79.384568,4,Café,Coffee Shop,Seafood Restaurant,Pizza Place,Opera House,Hotel,Plaza,Smoke Shop,Speakeasy,Fast Food Restaurant
36,M5J,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,4,Café,Park,Hotel,Plaza,IT Services,Deli / Bodega,Sports Bar,Sporting Goods Shop,Dance Studio,Japanese Restaurant
42,M5K,Toronto Dominion Centre / Design Exchange,43.647177,-79.381576,4,Coffee Shop,Café,Restaurant,Japanese Restaurant,Gym,Sandwich Place,Pub,Bookstore,Hotel,Pizza Place


## Observation Example
Cluster 1 focuses more on cafès as their most common venue. Cluster 4 as an example might be dominant with Coffee Shops, but classical cafés are found in all their lower levels of venues